In [34]:
import keras
import numpy as np
import tensorflow as tf
from tqdm import tqdm


In [3]:
%conda install regex

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::pandas==1.0.1=py36h0573a6f_0
  - defaults/noarch::jupyterlab==1.2.6=pyhf63ae98_0
  - defaults/linux-64::scikit-learn==0.22.1=py36hd81dba3_0
  - defaults/linux-64::python-language-server==0.31.7=py36_0
  - defaults/linux-64::bkcharts==0.2=py36_0
  - defaults/linux-64::nb_conda==2.2.1=py36_0
  - defaults/noarch::numpydoc==0.9.2=py_0
  - defaults/linux-64::pytest-arraydiff==0.3=py36h39e3cac_0
  - defaults/linux-64::bottleneck==1.3.2=py36heb32a55_0
  - defaults/noarch::sphinx==2.4.0=py_0
  - defaults/linux-64::pywavelets==1.1.1=py36h7b6447c_0
  - defaults/noarch::pytest-astropy==0.8.0=py_0
  - defaults/linux-64::numexpr==2.7.1=py36h423224d_0
  - defaults/noarch::anaconda-project==0.8.4=py_0
  - defaults/linux-64::nbconvert==5.6.1=py36_0
  - defaults/linux-64::h5py==2.10.0=py36h7918eee_0
  - defaults/linux-64::bokeh==1

# Prepare data

In [4]:
import unicodedata
import regex as re

def normalize_text(text):
    clean_text=text.lower()
    clean_text=re.sub(r"[^a-zA-Z0-9\säåö]"," ",clean_text)
    #clean_text=re.sub(r"[^\S\n]+"," ",clean_text)
    clean_text=re.sub(r"\s+"," ",clean_text)
    clean_text=re.sub(r"^\s","",clean_text)
    clean_text=re.sub(r"\s$","",clean_text)
    return clean_text

normalize_text("äåöjkjh jhjk j1hkj"),normalize_text("jFjh.jhjk,.,.ef jhkj "),normalize_text(" jkjh jhjk jhkj")

('äåöjkjh jhjk j1hkj', 'jfjh jhjk ef jhkj', 'jkjh jhjk jhkj')

In [35]:
import json 
import random
finnish_companies = json.load(open("finnish_kaggle_companies.json","r+"))
all_companies = json.load(open("all_kaggle_companies.json","r+"))

# companies 
names_arr=[normalize_text(company) for company in tqdm(all_companies)]
random.shuffle(names_arr)
names_text="\n".join(names_arr)

# finnish 
finnish_names_arr=[normalize_text(company) for company in tqdm(finnish_companies)]
random.shuffle(finnish_names_arr)
finnish_names_text="\n".join(finnish_names_arr)


characters=sorted(list(set(names_text+finnish_names_text)))

char_indices = dict((c, i) for i, c in enumerate(characters))
indices_char = dict((i, c) for i, c in enumerate(characters))

Tx=20






  0%|          | 0/7173427 [00:00<?, ?it/s]



  0%|          | 4798/7173427 [00:00<02:29, 47977.99it/s]



  0%|          | 9879/7173427 [00:00<02:26, 48792.89it/s]



  0%|          | 15359/7173427 [00:00<02:21, 50451.15it/s]



  0%|          | 20108/7173427 [00:00<02:24, 49522.91it/s]



  0%|          | 25677/7173427 [00:00<02:19, 51224.57it/s]



  0%|          | 31264/7173427 [00:00<02:15, 52534.57it/s]



  1%|          | 36508/7173427 [00:00<02:15, 52504.45it/s]



  1%|          | 41382/7173427 [00:00<02:19, 51031.46it/s]



  1%|          | 46718/7173427 [00:00<02:17, 51708.31it/s]



  1%|          | 52344/7173427 [00:01<02:14, 52993.32it/s]



  1%|          | 57982/7173427 [00:01<02:11, 53963.88it/s]



  1%|          | 63599/7173427 [00:01<02:10, 54607.01it/s]



  1%|          | 69168/7173427 [00:01<02:09, 54925.31it/s]



  1%|          | 74726/7173427 [00:01<02:08, 55117.56it/s]



  1%|          | 80338/7173427 [00:01<02:08, 55412.61it/s]



  1%|          | 8595

In [36]:
print("Characters: ","".join(characters))
print("Chars: ",len(characters))
print("Tx: ",Tx)

Characters:  
 0123456789abcdefghijklmnopqrstuvwxyzäåö
Chars:  41
Tx:  20


In [37]:
def str_to_vec(word):
    """
    Converts word to vec
    
    word -- string
    
    returns array of shape (Tx, len(chars))
    """
    x = np.zeros((len(word), len(characters)), dtype=np.bool)
    for t, char in enumerate(word):
        x[t, char_indices[char]] = 1
    return x
    
def vec_to_str(vec):
    """
    Converts vec to word
    
    vec -- array of shape (Tx, len(chars))
    
    """
    word=""
    for i in range(vec.shape[0]):
        word+=indices_char[np.argmax(vec[i])]
    return word

a=str_to_vec("hello world")
#print(a)
print(vec_to_str(a))

hello world


In [ ]:
def vectorization(text, stride, n_x, Tx):
    """
    Convert X and Y (lists) into arrays to be given to a recurrent neural network.
    
    Arguments:
    X -- 
    Y -- 
    Tx -- integer, sequence length
    
    Returns:
    x -- array of shape (m, Tx, len(chars))
    y -- array of shape (m, Tx, len(chars))
    """
    
    m = int((len(text)-Tx-1)/stride)
    x = np.zeros((m, Tx, n_x), dtype=np.bool)
    y = np.zeros((m, n_x), dtype=np.bool)
    
    for t, i in enumerate(tqdm(range(0,m*stride,stride))):
        fragment=text[i:i+Tx]
        pred=text[i+Tx+1]
        x[t, :, :] = str_to_vec(fragment)
        
        y[t, :] = str_to_vec(pred)
    return x, y 


X,Y=vectorization(names_text, 10, n_x=len(characters), Tx=Tx)
Xf,Yf=vectorization(finnish_names_text, 2, n_x=len(characters), Tx=Tx)
X.shape, Y.shape, Xf.shape, Yf.shape, 





  0%|          | 0/15348445 [00:00<?, ?it/s]



  0%|          | 6083/15348445 [00:00<04:12, 60827.74it/s]



  0%|          | 13735/15348445 [00:00<03:56, 64814.94it/s]



  0%|          | 21692/15348445 [00:00<03:43, 68632.50it/s]



  0%|          | 30632/15348445 [00:00<03:27, 73772.52it/s]



  0%|          | 38948/15348445 [00:00<03:20, 76356.59it/s]



  0%|          | 47650/15348445 [00:00<03:13, 79269.00it/s]



  0%|          | 57101/15348445 [00:00<03:03, 83296.94it/s]



  0%|          | 65626/15348445 [00:00<03:02, 83870.35it/s]



  0%|          | 74820/15348445 [00:00<02:57, 86137.59it/s]



  1%|          | 84824/15348445 [00:01<02:49, 89884.44it/s]



  1%|          | 94423/15348445 [00:01<02:46, 91630.56it/s]



  1%|          | 104472/15348445 [00:01<02:41, 94119.37it/s]



  1%|          | 114578/15348445 [00:01<02:38, 96098.72it/s]



  1%|          | 124759/15348445 [00:01<02:35, 97741.69it/s]



  1%|          | 134543/15348445 [00:01<02:37, 96548.29it/s]





In [ ]:
id_=343
vec_to_str(Xf[id_]),vec_to_str(np.array([Yf[id_]]))

# Model

In [40]:
tests=np.array([str_to_vec(a) for a in ["ap"]])

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    out = np.random.choice(range(len(characters)), p = probas.ravel())
    out=characters[out]
    return out




#[vec_to_str(res) for res in ]

In [41]:
def generate_output(model, text_start, length=5):
    generated = '\n'
    sentence = ('{0:\n>' + str(Tx) + '}').format(text_start).lower()
    generated += text_start 
    gens=0
    
    while gens<length and len(generated)<10000:
        x_pred = np.zeros((1, Tx, len(characters)))

        for t, char in enumerate(sentence):
            if char != '0':
                x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, temperature = 1.0)
        next_char = next_index

        generated += next_char
        sentence = sentence[1:] + next_char

        if next_char == '\n':
            gens+=1
    print(generated)
    
generate_output(model,"accent")


accentmnogam crdtnatoc
onlsea ote n  rnraehpot
asaswe egp
oedqn poatc ep
us nrndrl ygrug atmmtnagl



# Training

In [12]:
from keras import layers, models
from keras.layers import Dense, Input, LSTM,GRU


drp=0
model=models.Sequential()
model.add(GRU(128, input_shape=(Tx, len(characters)),return_sequences=True, dropout=drp))
#model.add(GRU(128,return_sequences=True, dropout=drp))
model.add(GRU(128,return_sequences=False, dropout=drp))
model.add(Dense(256,activation="relu"))
model.add(Dense(len(characters),activation="softmax"))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_1 (GRU)                  (None, 20, 128)           65280     
_________________________________________________________________
gru_2 (GRU)                  (None, 128)               98688     
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 41)                10537     
Total params: 207,529
Trainable params: 207,529
Non-trainable params: 0
_________________________________________________________________


In [13]:
from keras.callbacks import LambdaCallback

def on_epoch_end(epoch, logs):
    generate_output(model,"acc")

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)


In [24]:
model.compile(loss='categorical_crossentropy', optimizer="adam")

history=model.fit(X, Y, batch_size=2048, validation_split=0.1,  epochs=20, shuffle=True, callbacks=[print_callback])

Train on 13813600 samples, validate on 1534845 samples
Epoch 1/20
13813600/13813600 [==============================] - 936s 68us/step - loss: 2.1740 - val_loss: 2.1185

accmdxcpsresfrihm
asacrsse pyrcmdwrsoscuaol
oehxogeae
lgugran ecai l 
iiega ilao abkbajmpgrfns

Epoch 2/20
13813600/13813600 [==============================] - 934s 68us/step - loss: 2.0946 - val_loss: 2.0809

accapsac
oedbeekt gbt d   olrbn
ieniitmgetala  nsmge eoxsiec
aiatssbasi egp
rb vl idla pye

Epoch 3/20
  661504/13813600 [>.............................] - ETA: 14:35 - loss: 2.0742

KeyboardInterrupt: 

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer="adam")

history2=model.fit(X, Y, batch_size=2048, validation_split=0.1,  epochs=20, shuffle=False, callbacks=[print_callback])